In [1]:
import pandas as pd
from be_great import GReaT

/Users/magnus/Developer/ETH/SynthDataGen/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load CSV file
file_path = "../data/sample_level_2_100k.csv"
data = pd.read_csv(file_path, delimiter=';')

In [3]:
model = GReaT(llm='distilgpt2', batch_size=64,  epochs=10, fp16=False, use_mps_device=True) #should be fp16=True but MPS causes issues

In [4]:
print(f"data.shape: {data.shape}")
subset = data.sample(n=20000)
model.fit(subset)

data.shape: (100000, 11)


/Users/magnus/Developer/ETH/SynthDataGen/env/lib/python3.10/site-packages/transformers/training_args.py:2199: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
 16%|█▌        | 500/3130 [49:08<4:51:16,  6.64s/it] 

{'loss': 2.5139, 'grad_norm': 1.5517935752868652, 'learning_rate': 4.201277955271566e-05, 'epoch': 1.6}


 32%|███▏      | 1000/3130 [2:26:14<13:45:26, 23.25s/it]

{'loss': 2.3536, 'grad_norm': 1.4010268449783325, 'learning_rate': 3.402555910543131e-05, 'epoch': 3.19}


 48%|████▊     | 1500/3130 [5:15:20<20:33:37, 45.41s/it]  

{'loss': 2.309, 'grad_norm': 1.4866938591003418, 'learning_rate': 2.6038338658146967e-05, 'epoch': 4.79}


 52%|█████▏    | 1641/3130 [5:49:20<3:28:10,  8.39s/it]  

KeyboardInterrupt: 

In [10]:
# mps

synthetic_data = model.sample(n_samples=1000, device='mps', max_length=1000)
synthetic_data.head()

1045it [01:40, 10.40it/s]                         


,gender,salary,rent,age,canton,lat,lon,height,weight,z1,z2
0,Gender.MALE,0.000000,0.000000,16.0,BE,46.71612,7.64683,1.719398,54.669765,1.859191,7.74755
1,Gender.MALE,8117.453428,1817.835658,54.0,BE,46.90625,7.56954,1.726064,129.927350,1.967153,7.74935
2,Gender.MALE,4685.049381,929.367738,63.0,GE,46.81697,6.86167,1.786989,98.858122,0.539672,7.68232
3,Gender.MALE,4381.858388,821.943260,53.0,AG,47.20581,8.65502,1.732415,57.491808,1.564010,7.74226
4,Gender.FEMALE,4410.826351,813.982557,45.0,NE,46.49793,9.08689,1.807764,125.943285,1.962711,7.68472


In [8]:
# save model
model.save("great_model.pkl")

In [9]:
synthetic_data.shape

(100, 11)

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer, TVAESynthesizer, CopulaGANSynthesizer
from sdv.evaluation.single_table import evaluate_quality
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.single_column import KSComplement
from sdmetrics.column_pairs import CorrelationSimilarity
import time
import os

def load_and_prepare_data(file_path):
    """Load data and create metadata."""
    data = pd.read_csv(file_path, delimiter=';')
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    return data, metadata


def save_distribution_comparison(real_data, synthetic_data, column, title, folder):
    """Save distribution comparison for a specific column."""
    plt.figure(figsize=(12, 6))
    sns.histplot(real_data[column], kde=True, color="blue", alpha=0.5, label="Real")
    sns.histplot(synthetic_data[column], kde=True, color="red", alpha=0.5, label="Synthetic")
    plt.title(f'{title} - Distribution Comparison: {column}')
    plt.legend()
    plt.savefig(os.path.join(folder, f'{column}_distribution.png'))
    plt.close()

def save_correlation_heatmap(data, title, folder):
    """Save correlation heatmap for numeric columns only."""
    numeric_data = data.select_dtypes(include=[np.number])
    plt.figure(figsize=(12, 10))
    sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm')
    plt.title(f'{title} - Correlation Heatmap (Numeric Columns)')
    plt.savefig(os.path.join(folder, 'correlation_heatmap.png'))
    plt.close()

def compare_great_model(synthetic_data, real_data):
    """Compare the great model to real data."""
    # cut real data to same size as synthetic data
    real_data = real_data.sample(n=len(synthetic_data))
    
    print("Evaluating the great model...")
    evaluation = evaluate_synthesizer(real_data, synthetic_data)
    
    folder = 'plots_great_model'
    os.makedirs(folder, exist_ok=True)
    
    print("Saving plots for the great model...")
    for column in real_data.select_dtypes(include=[np.number]).columns:
        save_distribution_comparison(real_data, synthetic_data, column, 'Great Model', folder)
    save_correlation_heatmap(synthetic_data, 'Great Model', folder)
    
    print("Great Model Evaluation Results:")
    print(f"Quality Score: {evaluation['quality_report'].get_score():.4f}")
    print(f"Average KS Complement Score: {np.mean(list(evaluation['ks_scores'].values())):.4f}")
    print(f"Correlation Similarity Score: {evaluation['correlation_score']:.4f}")
    
    return {
        'synthetic_data': synthetic_data,
        'evaluation': evaluation
    }

# Main execution
file_path = "../data/sample_level_2_100k.csv"
model_path = "great_model.pkl"
results = compare_great_model(synthetic_data, data)

# Additional analysis or visualization
print("\nQuality Score:")
print(f"Great Model: {results['evaluation']['quality_report'].get_score():.4f}")

print("\nAverage KS Complement Score:")
print(f"Great Model: {np.mean(list(results['evaluation']['ks_scores'].values())):.4f}")

print("\nCorrelation Similarity Score:")
print(f"Great Model: {results['evaluation']['correlation_score']:.4f}")

# Plot quality scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['evaluation']['quality_report'].get_score()])
plt.title('Quality Score')
plt.ylabel('Score')
plt.savefig('great_model_quality_score.png')
plt.close()

# Plot average KS scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [np.mean(list(results['evaluation']['ks_scores'].values()))])
plt.title('Average KS Complement Score')
plt.ylabel('Score')
plt.savefig('great_model_ks_score.png')
plt.close()

# Plot correlation scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['evaluation']['correlation_score']])
plt.title('Correlation Similarity Score')
plt.ylabel('Score')
plt.savefig('great_model_correlation_score.png')
plt.close()

print("Comparison plots have been saved.")

Evaluating the great model...


TypeError: evaluate_synthesizer() missing 1 required positional argument: 'metadata'

In [12]:
model

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

def load_data(file_path):
    """Load real data."""
    return pd.read_csv(file_path, delimiter=';')

def calculate_ks_scores(real_data, synthetic_data):
    """Calculate Kolmogorov-Smirnov scores for numerical columns."""
    ks_scores = {}
    for column in real_data.select_dtypes(include=[np.number]).columns:
        ks_statistic, _ = stats.ks_2samp(real_data[column], synthetic_data[column])
        ks_scores[column] = 1 - ks_statistic  # Convert to similarity score
    return ks_scores

def calculate_correlation_similarity(real_data, synthetic_data):
    """Calculate correlation similarity between real and synthetic data."""
    real_corr = real_data.select_dtypes(include=[np.number]).corr()
    synthetic_corr = synthetic_data.select_dtypes(include=[np.number]).corr()
    return 1 - np.mean(np.abs(real_corr - synthetic_corr))

def calculate_categorical_similarity(real_data, synthetic_data):
    """Calculate similarity for categorical columns."""
    cat_scores = {}
    for column in real_data.select_dtypes(include=['object']).columns:
        real_dist = real_data[column].value_counts(normalize=True)
        synth_dist = synthetic_data[column].value_counts(normalize=True)
        cat_scores[column] = 1 - np.mean(np.abs(real_dist - synth_dist))
    return cat_scores

def save_distribution_comparison(real_data, synthetic_data, column, folder):
    """Save distribution comparison for a specific column."""
    plt.figure(figsize=(12, 6))
    
    if real_data[column].dtype == 'object' or real_data[column].dtype.name == 'category':
        # For categorical data
        real_dist = real_data[column].value_counts(normalize=True).sort_index()
        synth_dist = synthetic_data[column].value_counts(normalize=True).sort_index()
        
        # Combine categories to ensure both distributions have the same categories
        all_categories = sorted(set(real_dist.index) | set(synth_dist.index))
        real_dist = real_dist.reindex(all_categories, fill_value=0)
        synth_dist = synth_dist.reindex(all_categories, fill_value=0)
        
        x = range(len(all_categories))
        width = 0.35
        plt.bar([i - width/2 for i in x], real_dist.values, width, label='Real', alpha=0.5)
        plt.bar([i + width/2 for i in x], synth_dist.values, width, label='Synthetic', alpha=0.5)
        plt.xticks(x, all_categories, rotation=45, ha='right')
        plt.ylabel('Frequency')
    else:
        # For numerical data
        sns.kdeplot(data=real_data, x=column, label='Real', shade=True)
        sns.kdeplot(data=synthetic_data, x=column, label='Synthetic', shade=True)
        plt.ylabel('Density')
    
    plt.title(f'Distribution Comparison: {column}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(folder, f'{column}_distribution.png'))
    plt.close()

def save_correlation_heatmap(data, title, folder):
    """Save correlation heatmap for numeric columns only."""
    numeric_data = data.select_dtypes(include=[np.number])
    plt.figure(figsize=(12, 10))
    sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm')
    plt.title(f'{title} - Correlation Heatmap (Numeric Columns)')
    plt.tight_layout()
    plt.savefig(os.path.join(folder, 'correlation_heatmap.png'))
    plt.close()

def compare_great_model(real_data, synthetic_data):
    """Compare the great model to real data."""
    print("Evaluating the great model...")
    
    # Calculate KS scores for numerical columns
    ks_scores = calculate_ks_scores(real_data, synthetic_data)
    avg_ks_score = np.mean(list(ks_scores.values()))
    
    # Calculate correlation similarity for numerical columns
    correlation_score = calculate_correlation_similarity(real_data, synthetic_data)
    
    # Calculate similarity for categorical columns
    cat_scores = calculate_categorical_similarity(real_data, synthetic_data)
    avg_cat_score = np.mean(list(cat_scores.values()))
    
    folder = 'plots_great_model'
    os.makedirs(folder, exist_ok=True)
    
    print("Saving plots for the great model...")
    for column in real_data.columns:
        save_distribution_comparison(real_data, synthetic_data, column, folder)
    save_correlation_heatmap(synthetic_data, 'Great Model', folder)
    
    print("Great Model Evaluation Results:")
    print(f"Average KS Complement Score (Numerical): {avg_ks_score:.4f}")
    print(f"Correlation Similarity Score: {correlation_score:.4f}")
    print(f"Average Categorical Similarity Score: {avg_cat_score:.4f}")
    
    return {
        'ks_scores': ks_scores,
        'avg_ks_score': avg_ks_score,
        'correlation_score': correlation_score,
        'cat_scores': cat_scores,
        'avg_cat_score': avg_cat_score
    }

# Main execution
file_path = "../data/sample_level_2_100k.csv"
real_data = load_data(file_path)

# Assuming synthetic_data is already generated and available
# synthetic_data = ...

results = compare_great_model(real_data, synthetic_data)

# Additional analysis or visualization
print("\nDetailed KS Complement Scores (Numerical):")
for column, score in results['ks_scores'].items():
    print(f"{column}: {score:.4f}")

print("\nDetailed Categorical Similarity Scores:")
for column, score in results['cat_scores'].items():
    print(f"{column}: {score:.4f}")

# Plot average KS scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['avg_ks_score']])
plt.title('Average KS Complement Score (Numerical)')
plt.ylabel('Score')
plt.savefig('great_model_ks_score.png')
plt.close()

# Plot correlation scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['correlation_score']])
plt.title('Correlation Similarity Score')
plt.ylabel('Score')
plt.savefig('great_model_correlation_score.png')
plt.close()

# Plot average categorical similarity scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['avg_cat_score']])
plt.title('Average Categorical Similarity Score')
plt.ylabel('Score')
plt.savefig('great_model_cat_score.png')
plt.close()

print("Comparison plots have been saved.")

Evaluating the great model...
Saving plots for the great model...


/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/2371695473.py:57: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(data=real_data, x=column, label='Real', shade=True)
/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/2371695473.py:58: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(data=synthetic_data, x=column, label='Synthetic', shade=True)
/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/2371695473.py:57: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(data=real_data, x=column, label='Real', shade=True)
/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/2371695473

Great Model Evaluation Results:
Average KS Complement Score (Numerical): 0.8773
Correlation Similarity Score: 0.8852
Average Categorical Similarity Score: 0.9739

Detailed KS Complement Scores (Numerical):
salary: 0.9293
rent: 0.8923
age: 0.8466
lat: 0.8767
lon: 0.8041
height: 0.9039
weight: 0.8674
z1: 0.8092
z2: 0.9667

Detailed Categorical Similarity Scores:
gender: 0.9619
canton: 0.9859
Comparison plots have been saved.


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

def save_distribution_comparison(real_data, synthetic_data, column, folder):
    """Save distribution comparison for a specific column."""
    plt.figure(figsize=(12, 6))
    
    if real_data[column].dtype == 'object' or real_data[column].dtype.name == 'category':
        # For categorical data
        real_dist = real_data[column].value_counts(normalize=True).sort_index()
        synth_dist = synthetic_data[column].value_counts(normalize=True).sort_index()
        
        # Combine categories to ensure both distributions have the same categories
        all_categories = sorted(set(real_dist.index) | set(synth_dist.index))
        real_dist = real_dist.reindex(all_categories, fill_value=0)
        synth_dist = synth_dist.reindex(all_categories, fill_value=0)
        
        x = range(len(all_categories))
        width = 0.35
        plt.bar([i - width/2 for i in x], real_dist.values, width, label='Real', alpha=0.5)
        plt.bar([i + width/2 for i in x], synth_dist.values, width, label='Synthetic', alpha=0.5)
        plt.xticks(x, all_categories, rotation=45, ha='right')
        plt.ylabel('Frequency')
    else:
        # For numerical data
        sns.kdeplot(data=real_data, x=column, label='Real', shade=True)
        sns.kdeplot(data=synthetic_data, x=column, label='Synthetic', shade=True)
        plt.ylabel('Density')
    
    plt.title(f'Distribution Comparison: {column}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(folder, f'{column}_distribution.png'))
    plt.close()

def compare_great_model(real_data, synthetic_data):
    """Compare the great model to real data."""
    print("Evaluating the great model...")
    
    # Calculate KS scores for numerical columns
    ks_scores = calculate_ks_scores(real_data, synthetic_data)
    avg_ks_score = np.mean(list(ks_scores.values()))
    
    # Calculate correlation similarity for numerical columns
    correlation_score = calculate_correlation_similarity(real_data, synthetic_data)
    
    # Calculate similarity for categorical columns
    cat_scores = calculate_categorical_similarity(real_data, synthetic_data)
    avg_cat_score = np.mean(list(cat_scores.values()))
    
    folder = 'plots_great_model'
    os.makedirs(folder, exist_ok=True)
    
    print("Saving plots for the great model...")
    for column in real_data.columns:
        save_distribution_comparison(real_data, synthetic_data, column, folder)
    save_correlation_heatmap(synthetic_data, 'Great Model', folder)
    
    print("Great Model Evaluation Results:")
    print(f"Average KS Complement Score (Numerical): {avg_ks_score:.4f}")
    print(f"Correlation Similarity Score: {correlation_score:.4f}")
    print(f"Average Categorical Similarity Score: {avg_cat_score:.4f}")
    
    return {
        'ks_scores': ks_scores,
        'avg_ks_score': avg_ks_score,
        'correlation_score': correlation_score,
        'cat_scores': cat_scores,
        'avg_cat_score': avg_cat_score
    }

# Main execution
file_path = "../data/sample_level_2_100k.csv"
real_data = load_data(file_path)

# Assuming synthetic_data is already generated and available
# synthetic_data = ...

results = compare_great_model(real_data, synthetic_data)

# Additional analysis or visualization
print("\nDetailed KS Complement Scores (Numerical):")
for column, score in results['ks_scores'].items():
    print(f"{column}: {score:.4f}")

print("\nDetailed Categorical Similarity Scores:")
for column, score in results['cat_scores'].items():
    print(f"{column}: {score:.4f}")

# Plot average KS scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['avg_ks_score']])
plt.title('Average KS Complement Score (Numerical)')
plt.ylabel('Score')
plt.savefig('great_model_ks_score.png')
plt.close()

# Plot correlation scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['correlation_score']])
plt.title('Correlation Similarity Score')
plt.ylabel('Score')
plt.savefig('great_model_correlation_score.png')
plt.close()

# Plot average categorical similarity scores
plt.figure(figsize=(10, 6))
plt.bar(['Great Model'], [results['avg_cat_score']])
plt.title('Average Categorical Similarity Score')
plt.ylabel('Score')
plt.savefig('great_model_cat_score.png')
plt.close()

print("Comparison plots have been saved.")

Evaluating the great model...
Saving plots for the great model...


/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/3434630740.py:30: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(data=real_data, x=column, label='Real', shade=True)
/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/3434630740.py:31: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(data=synthetic_data, x=column, label='Synthetic', shade=True)
/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/3434630740.py:30: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(data=real_data, x=column, label='Real', shade=True)
/var/folders/q0/hy1vcj9j02qdx_zx_2mqwvhm0000gn/T/ipykernel_66040/3434630740

Great Model Evaluation Results:
Average KS Complement Score (Numerical): 0.8773
Correlation Similarity Score: 0.8852
Average Categorical Similarity Score: 0.9739

Detailed KS Complement Scores (Numerical):
salary: 0.9293
rent: 0.8923
age: 0.8466
lat: 0.8767
lon: 0.8041
height: 0.9039
weight: 0.8674
z1: 0.8092
z2: 0.9667

Detailed Categorical Similarity Scores:
gender: 0.9619
canton: 0.9859
Comparison plots have been saved.
